In [1]:
# Установка nodejs и localtunnel
!curl -sL https://deb.nodesource.com/setup_14.x | bash -
!apt install -y nodejs
!npm install -g localtunnel

# Установка необходимых библиотек Python
!pip install flask pyngrok scikit-learn pandas



                              DEPRECATION WARNING                            

  Node.js 14.x is no longer actively supported!

  You will not receive security or critical stability updates for this version.

  You should migrate to a supported version of Node.js as soon as possible.
  Use the installation script that corresponds to the version of Node.js you
  wish to install. e.g.

   * https://deb.nodesource.com/setup_16.x — Node.js 16 "Gallium"
   * https://deb.nodesource.com/setup_18.x — Node.js 18 LTS "Hydrogen" (recommended)
   * https://deb.nodesource.com/setup_19.x — Node.js 19 "Nineteen"
   * https://deb.nodesource.com/setup_20.x — Node.js 20 "Iron" (current)

  Please see https://github.com/nodejs/Release for details about which
  version may be appropriate for you.

  The NodeSource Node.js distributions repository contains
  information both about supported versions of Node.js and supported Linux
  distributions. To learn more about usage, see the repository:
    https:/

In [ ]:
pip install flask pyngrok

In [1]:
import pandas as pd
from flask import Flask, request, jsonify, render_template
import pickle
import threading
import time
#from pyngrok import ngrok
import subprocess
import requests
import os
import shutil


In [2]:
os.makedirs('templates', exist_ok=True)

# Копируем ваши файлы из sample_data в папку templates
try:
    shutil.copy2('D:/maga_poly/ML_3_sem/ml_polytech_3_sem/03/index.html', 'templates/index.html')
    shutil.copy2('D:/maga_poly/ML_3_sem/ml_polytech_3_sem/03/result.html', 'templates/result.html')
    print("HTML файлы успешно скопированы из sample_data в templates")
except FileNotFoundError as e:
    print(f"Ошибка: {e}")
    print("Создаем базовые шаблоны...")

    # Создаем базовые шаблоны, если ваши файлы не найдены
    with open('templates/index.html', 'w') as f:
        f.write('''
<!DOCTYPE html>
<html>
<head>
    <title>ML Service</title>
</head>
<body>
    <h2>Предсказание с помощью ML модели</h2>
    <form action="/predict" method="post">
        <label>Введите данные для предсказания (через запятую):</label><br>
        <input type="text" name="features" size="50" required><br><br>
        <input type="submit" value="Предсказать">
    </form>
</body>
</html>
''')

    with open('templates/result.html', 'w') as f:
        f.write('''
<!DOCTYPE html>
<html>
<head>
    <title>Результат предсказания</title>
</head>
<body>
    <h2>Результат предсказания</h2>
    <p>{{ prediction }}</p>
    <br>
    <a href="/">Назад</a>
</body>
</html>
''')


HTML файлы успешно скопированы из sample_data в templates


In [3]:
# Загрузка или обучение модели
try:
    # Попробуйте загрузить вашу модель
    with open('D:/maga_poly/ML_3_sem/ml_polytech_3_sem/03/model1.pkl', 'rb') as f:
        model = pickle.load(f)
    print("Ваша модель успешно загружена")
except:
    # Если модели нет, обучим простую модель для примера
    from sklearn.datasets import load_iris
    from sklearn.ensemble import RandomForestClassifier

    print("Создаем демо-модель (ирисы Фишера)...")
    iris = load_iris()
    X, y = iris.data, iris.target

    model = RandomForestClassifier()
    model.fit(X, y)

    with open('your_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    print("Демо-модель создана и сохранена")


Ваша модель успешно загружена


In [ ]:
# def process_input_data(form_data):
#     processed_features = {}

#     for key, value in form_data.items():
#         if key.startswith('feature'):
#             # Обрабатываем NULL значения
#             if value in ['', 'NULL', 'null', 'None', 'none']:
#                 processed_features[key] = None
#             else:
#                 # Пытаемся преобразовать в число
#                 try:
#                     processed_features[key] = float(value)
#                 except ValueError:
#                     # Если не получается, оставляем как строку
#                     processed_features[key] = value

#     return processed_features

In [ ]:

# # Функция для обработки входящих данных
# def process_input_data(form_data):

#     processed_features = {}

#     # Проверяем, есть ли поле 'features' с данными через запятую
#     if 'features' in form_data and form_data['features'].strip():
#         # Обрабатываем строку с значениями через запятую
#         features_str = form_data['features']
#         print(f"Обнаружена строка с признаками: {features_str}")

#         # Разделяем по запятым и обрабатываем каждое значение
#         values = features_str.split(',')
#         features_list = []

#         for i, value in enumerate(values, 1):
#             value = value.strip()  # Убираем пробелы

#             # Обрабатываем NULL значения
#             if value in ['', 'NULL', 'null', 'None', 'none']:
#                 features_list.append(0.0)
#                 processed_features[f'feature{i}'] = 0.0
#             else:
#                 # Пытаемся преобразовать в число
#                 try:
#                     # Заменяем запятые на точки для корректного преобразования float
#                     value = value.replace(',', '.')
#                     num_value = float(value)
#                     features_list.append(num_value)
#                     processed_features[f'feature{i}'] = num_value
#                 except ValueError:
#                     # Если не получается, используем 0 и логируем
#                     print(f"Не удалось преобразовать '{value}' в число, используем 0")
#                     features_list.append(0.0)
#                     processed_features[f'feature{i}'] = 0.0

#         # Добавляем список признаков для удобства
#         processed_features['features_list'] = features_list
#         print(f"Преобразовано {len(features_list)} признаков")

#     else:
#         # Старый способ: обрабатываем отдельные поля feature1, feature2, ...
#         for key, value in form_data.items():
#             if key.startswith('feature') and not key == 'features':
#                 # Обрабатываем NULL значения
#                 if value in ['', 'NULL', 'null', 'None', 'none']:
#                     processed_features[key] = 0.0
#                 else:
#                     # Пытаемся преобразовать в число
#                     try:
#                         # Заменяем запятые на точки для корректного преобразования float
#                         value = value.replace(',', '.')
#                         processed_features[key] = float(value)
#                     except ValueError:
#                         # Если не получается, используем 0 и логируем
#                         print(f"Не удалось преобразовать '{value}' в число, используем 0")
#                         processed_features[key] = 0.0

#         # Создаем список признаков в правильном порядке
#         features_list = []
#         i = 1
#         while f'feature{i}' in processed_features:
#             features_list.append(processed_features[f'feature{i}'])
#             i += 1

#         processed_features['features_list'] = features_list

#     return processed_features

# def prepare_features_for_model(features_dict):
#     # Используем features_list если он есть, иначе собираем из отдельных feature*
#     if 'features_list' in features_dict:
#         features_list = features_dict['features_list']
#     else:
#         # Собираем признаки в правильном порядке
#         features_list = []
#         i = 1
#         while f'feature{i}' in features_dict:
#             features_list.append(features_dict[f'feature{i}'])
#             i += 1
#     print(1)
#     # Определяем имена столбцов
#     num_features = len(features_list)
#     feature_columns = ["AGREEMENT_RK","AGE","SOCSTATUS_WORK_FL","SOCSTATUS_PENS_FL","GENDER","CHILD_TOTAL","DEPENDANTS","PERSONAL_INCOME","REG_FACT_FL",
#                        "FACT_POST_FL","REG_POST_FL","REG_FACT_POST_FL","REG_FACT_POST_TP_FL","FL_PRESENCE_FL","OWN_AUTO","AUTO_RUS_FL",
#                        "HS_PRESENCE_FL","COT_PRESENCE_FL","GAR_PRESENCE_FL","LAND_PRESENCE_FL","CREDIT","TERM","FST_PAYMENT","DL_DOCUMENT_FL",
#                        "GPF_DOCUMENT_FL","FACT_LIVING_TERM","WORK_TIME","FACT_PHONE_FL","REG_PHONE_FL","GEN_PHONE_FL","LOAN_NUM_TOTAL","LOAN_NUM_CLOSED","LOAN_NUM_PAYM","LOAN_DLQ_NUM","LOAN_MAX_DLQ","LOAN_AVG_DLQ_AMT","LOAN_MAX_DLQ_AMT","PREVIOUS_CARD_NUM_UTILIZED"]

#     # Создаем DataFrame с правильными именами столбцов
#     features_df = pd.DataFrame([features_list], columns=feature_columns)
#     print(2)
#     # Гарантируем, что все данные будут числовыми
#     for col in features_df.columns:
#         features_df[col] = pd.to_numeric(features_df[col], errors='coerce').fillna(0.0)
#     print(3)
#     print(f"Подготовленный DataFrame:")
#     print(features_df.head())
#     print(f"Форма данных: {features_df.shape}")
#     print(f"Типы данных: {features_df.dtypes}")

#     return features_df


In [4]:
import numpy as np

def ValuePredictor(to_predict_list):
    to_predict = np.array(to_predict_list).reshape(1,len(to_predict_list))
    print(to_predict)
    #model = pickle.load(open("model1.pkl","rb"))
    result = model.predict_proba(to_predict)[:, 1]
    print(result)
    return result[0]

In [5]:
# Создание Flask-приложения
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # # Получаем данные из формы
        # raw_features = process_input_data(request.form)
        # print(raw_features)
        # prepared_data = prepare_features_for_model(raw_features)
        # print(prepared_data)
        features_str = request.form['features']
        print(f"Не удалось преобразовать 1 '{features_str}' в float")
        str_lst = features_str.split(sep = ',')
        features = []
        for i in str_lst:
          features.append(float(i.strip()))
        print(f"Удалось преобразовать '{features}' в float")
        print(features)

        #features = [float(x.strip()) for x in features_str.split(',')]
        # # Делаем предсказание
        # #prediction = model.predict(raw_features)

        # # Форматируем результат

        # #prediction = model.predict(raw_features)
        # #prediction = model.predict_proba(raw_features)[:, 1]
        # #prediction = float(prediction)
        # prediction_proba = model.predict_proba(prepared_data)
        # if prediction_proba.shape[1] > 1:
        #   probability = prediction_proba[0, 1]
        # else:
        #   probability = prediction_proba[0, 0]
        # to_predict_list = request.form.to_dict()
        # to_predict_list=list(to_predict_list.values())

        #features = list(map(float, to_predict_list))

        probability = ValuePredictor(features)
        # Форматируем результат
        result = f"Предсказание: {probability:.6f} "
        return render_template('result.html', prediction=result)

    except Exception as e:
        return f"Ошибка: {str(e)}"

In [6]:
# Функция для запуска Flask
def run_flask():
    app.run(host='0.0.0.0', port=5000)

# Функция для запуска LocalTunnel
#def run_localtunnel():
    # Ждем немного, чтобы Flask успел запуститься
    time.sleep(3)

    # Получаем публичный IP для пароля
    try:
        ip_response = requests.get('https://api.ipify.org')
        public_ip = ip_response.text
        print(f"Пароль для доступа: {public_ip}")
    except:
        print("Не удалось получить IP автоматически")
        public_ip = "unknown"

    # Запускаем LocalTunnel с дополнительными параметрами
    process = subprocess.Popen(
        ['lt', '--port', '5000', '--print-requests', '--local-host', '127.0.0.1'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )

    # Читаем вывод для получения URL
    for line in process.stdout:
        if 'your url is:' in line:
            url = line.split('your url is: ')[1].strip()
            print(f"\n=== Ваш веб-сервис доступен по адресу: {url} ===")
            print(f"=== Пароль для доступа: {public_ip} ===")
            print("Скопируйте URL и откройте в браузере")
            print("При запросе пароля введите указанный выше IP-адрес")
            break
# Запускаем Flask в отдельном потоке
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Запускаем LocalTunnel
print("Запускаем LocalTunnel...")
#run_localtunnel()

# Держим ячейку активной
while True:
    time.sleep(1)

Запускаем LocalTunnel...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.210.1.0:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Sep/2025 19:36:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2025 19:36:09] "POST /predict HTTP/1.1" 200 -


Не удалось преобразовать 1 '-0.051883920891956994,-0.1674258895778759,-0.07417629889299825,-0.09942313670820434,0.07081625940673057,0.023659995639533627,-0.05396868352813304,0.3651161178949416,-0.2679999405919769,0.0,-0.3528475870451933,-0.26465465670761,0.051857683227804534,0.058618489168338395,0.02383212077592562,0.008804612084869051,-0.013331602191203013,0.0,0.0,0.0,0.030182371332638677,0.11664443480708964,-0.09913322458725772,0.0,0.07420939217294543,0.00997488723305937,-0.13079200004274438,0.09592780923272025,-0.00017983562132401003,-0.07546681281886483,0.20107852419629302,0.3349707834723416,-0.07066969247030434,0.09688463426933629,0.09688463426933629,0.09711684397445786,0.09678110450512999,0.0' в float
Удалось преобразовать '[-0.051883920891956994, -0.1674258895778759, -0.07417629889299825, -0.09942313670820434, 0.07081625940673057, 0.023659995639533627, -0.05396868352813304, 0.3651161178949416, -0.2679999405919769, 0.0, -0.3528475870451933, -0.26465465670761, 0.051857683227804534

127.0.0.1 - - [25/Sep/2025 19:36:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2025 19:36:33] "POST /predict HTTP/1.1" 200 -


Не удалось преобразовать 1 '-0.051883920891956994,-0.1674258895778759,-0.07417629889299825,-0.09942313670820434,0.07081625940673057,0.023659995639533627,-0.05396868352813304,0.3651161178949416,-0.2679999405919769,0.0,-0.3528475870451933,-0.26465465670761,0.051857683227804534,0.058618489168338395,0.02383212077592562,0.008804612084869051,-0.013331602191203013,0.0,0.0,0.0,0.030182371332638677,0.11664443480708964,-0.09913322458725772,0.0,0.07420939217294543,0.00997488723305937,-0.13079200004274438,0.09592780923272025,-0.00017983562132401003,-0.07546681281886483,0.20107852419629302,0.3349707834723416,-0.07066969247030434,0.09688463426933629,0.09688463426933629,0.09711684397445786,0.09678110450512999,0.0' в float
Удалось преобразовать '[-0.051883920891956994, -0.1674258895778759, -0.07417629889299825, -0.09942313670820434, 0.07081625940673057, 0.023659995639533627, -0.05396868352813304, 0.3651161178949416, -0.2679999405919769, 0.0, -0.3528475870451933, -0.26465465670761, 0.051857683227804534

KeyboardInterrupt: 